# Terminal Link Optimization

This is the main ipython script for the terminal link optimization algorithm.
I will update the documentation accordingly once the project has progressed further.

## Import packages

In [0]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
device = 'cuda' if torch.cuda.is_available() else 'cpu'

## Make simulation of satellite and customers

Lets first start by simulating customers on the ground. This will essentially 
end up being a straight line with dots (representing customers) being randomly 
distributed along the line. The line will be spaced according to some 
arbitrary time metric. Each customer will have (for now) 4 attributes which 
describe the "importantness" of each customer.

In [0]:
"""
Lets make an array containing the physical location of some simulated customers.
The array will have the shape (N_time_segments,length_of_time_segment)

customer data array `customer_arr` will be of shape (number of training orbits, number of customers 
in orbit, number of features describing priority of customers)
"""

# training set hyperparameters
num_customers = 100 # total number of customers on planet
num_orbits = 10000  # total number of training samples (in this case orbits)
max_dist = 10000    # maximum distance allowed for customers
customer_arr = np.zeros((num_orbits,num_customers,5)) # Define emtpy array to contain customer data

for i in range(num_orbits):
    # Make customer index labels
    customer_arr[i,:,0] = np.arange(start=0,stop=num_customers,step=1)

    # Define random locations of customers on a line defining the planet
    customer_arr[i,:,1] = np.random.randint(low=0,high=max_dist,size=(num_customers))
    
    # We will Define 4 variables which describe the `importantness` of each customer

    # Assign a random customer important factor. 0 == low importance, 1 == high importance
    customer_arr[i,:,2] = np.random.uniform(low=0.0,high=1.0,size=(num_customers))

    # Assign a random customer weather factor. 0 == high cloud coverage, 1 == low cloud coverage
    customer_arr[i,:,3] = np.random.uniform(low=0.0,high=1.0,size=(num_customers))



## Choose optimal order in which to distribute keys (simple approach)

This is an incredibly simple algorithm which only takes into account 
the importantness and weather factors for each customer. It does not 
take into account the total time customers have been waiting for 
a key to be distributed to them.

In [0]:
# Loop over all available customers within line-of-sight (this is by default set to num_customers)
# This is an example using only one training orbit.
for i in range(num_customers):
    customer_arr[0,i,4] = customer_arr[0,i,2] * customer_arr[0,i,3]

customer_prob_list_idx = np.argsort(customer_arr[0,:,4])[::-1]
customer_prob_list = customer_arr[0,customer_prob_list_idx,0]

In [10]:
# Print out ordered list of customers to distribute keys to
customer_prob_list

array([19., 46.,  0., 30., 45., 95., 74., 32., 76., 65., 53., 43., 78.,
       40., 70.,  5.,  3., 21., 84., 73., 97., 81.,  8., 52., 79., 96.,
       54., 77., 13., 67., 93.,  1., 31., 49., 92., 99., 24., 50., 35.,
       82., 15., 75., 44., 25., 14., 69.,  9., 58., 37., 56., 63., 42.,
       86., 28., 27., 90., 72., 48., 71., 26., 57., 55., 20., 51., 39.,
       61., 85., 62., 18., 38., 83., 66., 22., 29.,  6., 36., 64.,  4.,
       98., 41., 11., 68., 16., 12.,  7., 59., 33., 87., 60., 47.,  2.,
       80., 10., 94., 23., 89., 17., 34., 88., 91.])

## Neural Network Approach (complicated approach)

I've written some pseudo code down on a piece of paper. Not guranteeing that 
this will actually work in practice, but I will make my first attempt of this here.

### Define network archetecture

In [11]:
# This approach uses the Pytorch neural network library

# neural network hyperparameters
number_epochs = 100
batch_size = 64

# Define network archetecture (convolutional neural network)
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, (1,3), stride=1).double()
        self.conv2 = nn.Conv2d(32, 64, (1,3), stride=1).double()
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(6144, 120).double()
        self.fc2 = nn.Linear(120, 84).double()
        self.fc3 = nn.Linear(84, num_customers).double() # one node for predicted probability

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.relu(self.conv1(x))
        # If the size is a square you can only specify a single number
        x = F.relu(self.conv2(x))
        x = x.view(-1, self.num_flat_features(x))
        # Two fully-connected hidden layers and one output layer
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
net.to(device)
print(net)

params = list(net.parameters())

criterion = nn.MSELoss()

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.001)

Net(
  (conv1): Conv2d(3, 32, kernel_size=(1, 3), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(1, 3), stride=(1, 1))
  (fc1): Linear(in_features=6144, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=100, bias=True)
)


In [0]:
def custom_loss(output, target, batch_feature_data):
    """ This is a custom loss function which 
    takes as input the raw output from the neural 
    network and returns a loss which attempts to 
    minimize the number of keys not distributed 
    within the alloted amount of time.
    """
    
    def my_model(output, batch_feature_data):
        """ This is a model which will iterate 
        over the predicted ordering of keys to be 
        distributed. There will be a penalty 
        associated with any keys which are predicted to 
        be distributed outside of the obersvation window.
        """

        # Assume 5 seconds to distribute key to customer downstream
        # Array to store the calculated number of keys lying outside of time alloted
        new_output = np.zeros((batch_feature_data.shape[0],1))

        # Iterate over all samples in batch
        for i in range(batch_feature_data.shape[0]):

          # Sort predicted output sample by probability from high to low
          tmp_output = np.sort(output[i,:].detach().numpy())[::-1]

          # Iterate through probability preds for 1 sample from high to low
          dist_cnt = 0              # Initialize distance traveled to be zero
          sat_over_horizon = False  # If True, satellite has started going out of reach
          for j in range(tmp_output.shape[0]):

            # Once satellite goes over horizon, start dropping customers
            if dist_cnt >= max_dist:
              dist_cnt = 0             # Reset distance counter
              sat_over_horizon = True  # set satellite to now be over horizon
            
            if sat_over_horizon == True:
              if batch_feature_data[i,j,0] < dist_cnt:
                # Add penalty associated with importantness and weather
                new_output[i,0] += (batch_feature_data[i,j,1] + 
                                    batch_feature_data[i,j,2])

            # Add a distance traveled by satellite
            dist_cnt += 200
    
        return torch.tensor(new_output, requires_grad=True)
    
    # Get number of keys predicted to be outside of target window
    pred_model_score = my_model(output, batch_feature_data)

    print(pred_model_score.shape,target.shape)
    
    loss = torch.mean((pred_model_score - torch.tensor(target, requires_grad=True))**2)
    return loss

### Run network over entire training set

In [0]:
# training features: location, importance, weather
X_train = customer_arr[:,:,1:4]
Y_train = np.zeros((num_orbits,1))

In [28]:
running_loss = 0.0
train_split = num_orbits

dset_train = TensorDataset(torch.tensor(X_train), torch.tensor(Y_train))
dataloader = DataLoader(dset_train, batch_size=batch_size,
                        shuffle=True)

# Iterate over entire training set
for epoch_num in range(number_epochs):
    for i_batch, sampled_batch in enumerate(dataloader):
        # in your training loop:
        if (train_split - (batch_size * i_batch)) < batch_size:
            break
        optimizer.zero_grad()   # zero the gradient buffers
        output = net(sampled_batch[0].reshape(batch_size,3,1,num_customers).to(device))
        loss = custom_loss(output, Y_train[i_batch:i_batch+batch_size].reshape(batch_size,1), sampled_batch[0])
        loss.backward()
        optimizer.step()    # Does the update to the network    
        
    
    
#    print('Training epoch %d/%d' % (epoch_num+1,number_epochs))
    # print statistics
#    running_loss += loss.item()
#    if i % int(X_train.shape[0]/batch_size) == (int(X_train.shape[0]/batch_size) - 1):    # print every 2000 mini-batches
#        print('loss: %.3f' %
#                (running_loss / 10))
#        running_loss = 0.0
#        epoch_num += 1
#        print('Training epoch %d/%d' % (epoch_num,number_epochs))

torch.Size([64, 1]) (64, 1)
torch.Size([64, 1]) (64, 1)
torch.Size([64, 1]) (64, 1)
torch.Size([64, 1]) (64, 1)
torch.Size([64, 1]) (64, 1)
torch.Size([64, 1]) (64, 1)
torch.Size([64, 1]) (64, 1)
torch.Size([64, 1]) (64, 1)
torch.Size([64, 1]) (64, 1)
torch.Size([64, 1]) (64, 1)
torch.Size([64, 1]) (64, 1)
torch.Size([64, 1]) (64, 1)
torch.Size([64, 1]) (64, 1)
torch.Size([64, 1]) (64, 1)
torch.Size([64, 1]) (64, 1)
torch.Size([64, 1]) (64, 1)
torch.Size([64, 1]) (64, 1)
torch.Size([64, 1]) (64, 1)
torch.Size([64, 1]) (64, 1)
torch.Size([64, 1]) (64, 1)
torch.Size([64, 1]) (64, 1)
torch.Size([64, 1]) (64, 1)
torch.Size([64, 1]) (64, 1)
torch.Size([64, 1]) (64, 1)
torch.Size([64, 1]) (64, 1)
torch.Size([64, 1]) (64, 1)
torch.Size([64, 1]) (64, 1)
torch.Size([64, 1]) (64, 1)
torch.Size([64, 1]) (64, 1)
torch.Size([64, 1]) (64, 1)
torch.Size([64, 1]) (64, 1)
torch.Size([64, 1]) (64, 1)
torch.Size([64, 1]) (64, 1)
torch.Size([64, 1]) (64, 1)
torch.Size([64, 1]) (64, 1)
torch.Size([64, 1]) 

KeyboardInterrupt: ignored

In [0]:
""